In [ ]:
%load_ext autoreload
%autoreload 2
is_cuda = True

# General setup and perturbation

In [1]:
%pip install -e .

Obtaining file:///Users/smcaleese/Documents/masters-thesis-code/polyjuice
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for polyjuice_nlp (pyproject.toml) ... done
  Created wheel for polyjuice_nlp: filename=polyjuice_nlp-0.1.5-0.editable-py3-none-any.whl size=5964 sha256=a0fb4cd0cf088898c1a82113968350c6e875839df99f584e86de27b83ee05993
  Stored in directory: /private/var/folders/t6/_lt6g5116z9f5127kxxf3qgc0000gn/T/pip-ephem-wheel-cache-ai1zlcuo/wheels/25/ab/5a/2c39cb2ced826c744df003583a7e2691ec72e79dc71b9ba517
Successfully built polyjuice_nlp
Note: you may need to restart the kernel to use updated packages.


In [2]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 10.5 MB/s eta 0:00:0000:010:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [3]:
from transformers import pipeline

# Initialize the text generation pipeline
generator = pipeline('text-generation', model='gpt2')

# Define the prompt
prompt = "Once upon a time"

# Generate text
generated_texts = generator(prompt, max_length=50, num_return_sequences=3, temperature=0.7, top_k=50, top_p=0.9)

# Print the generated texts
for i, generated_text in enumerate(generated_texts):
    print(f"Generated Text {i + 1}: {generated_text['generated_text']}")


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Text 1: Once upon a time, I had been reading an article about "How to Be a Good Customer" and "The Most Important Customer" by Mark Twain. I thought it was a very good book, and I was so excited to read it.

Generated Text 2: Once upon a time, I was very much afraid that my husband and I would have to go to work together. I was worried that if he did not have the money he would be forced to go to work for us. I was also worried that
Generated Text 3: Once upon a time, the planet was the center of galactic life. But the ancient inhabitants of the Milky Way had been wiped out by a mysterious force that had been unleashed upon the galaxy by a mysterious force known as the Vast Reaper. The V


In [5]:
from transformers import AutoTokenizer, AutoModelWithLMHead
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM

model_path = "uw-hai/polyjuice"
generator = pipeline("text-generation", 
    model=AutoModelForCausalLM.from_pretrained(model_path), 
    tokenizer=AutoTokenizer.from_pretrained(model_path),
    framework="pt")

prompt_text = "A dog is embraced by the woman. <|perturb|> [negation] A dog is [BLANK] the woman."
generator(prompt_text, num_beams=3, num_return_sequences=3)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'A dog is embraced by the woman. <|perturb|> [negation] A dog is [BLANK] the woman. [SEP] not being cradled by [ANSWER] '},
 {'generated_text': 'A dog is embraced by the woman. <|perturb|> [negation] A dog is [BLANK] the woman. [SEP] not [ANSWER] '},
 {'generated_text': 'A dog is embraced by the woman. <|perturb|> [negation] A dog is [BLANK] the woman. [SEP] not being cradled by a person [ANSWER] '}]

In [1]:
# initiate a wrapper.
# model path is defaulted to our portable model:
# https://huggingface.co/uw-hai/polyjuice
# No need to change this unless you are using customized model
from polyjuice import Polyjuice
pj = Polyjuice(model_path="uw-hai/polyjuice", is_cuda=True)

/Users/smcaleese/Documents/masters-thesis-code/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# the base sentence
text = "It is great for kids."

# perturb the sentence with one line:
# When running it for the first time, the wrapper will automatically
# load related models, e.g. the generator and the perplexity filter.
perturbations = pj.perturb(text)
perturbations

INFO:polyjuice.polyjuice_wrapper:Setup Polyjuice.
INFO:polyjuice.polyjuice_wrapper:Setup SpaCy processor.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/Users/smcaleese/Documents/masters-thesis-code/.venv/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:554: UserWarning: `num_beams` is set to None - defaulting to 1.
  warnings.warn("`num_beams` is set to None - defaulting to 1.", UserWarning)
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_tok

['It is great for any kids.', 'It is bad for kids.', 'Not is great for kids.']

In [7]:
# To perturb with more controls,

perturbations = pj.perturb(
    orig_sent=text,
    # can specify where to put the blank. Otherwise, it's automatically selected.
    # Can be a list or a single sentence.
    # blanked_sent=["It is [BLANK] for kids.", "It is great for [BLANK]."],
    # can also specify the ctrl code (a list or a single code.)
    # The code should be from 'resemantic', 'restructure', 'negation', 'insert', 'lexical', 'shuffle', 'quantifier', 'delete'.
    ctrl_code="negation",
    # Customzie perplexity score. 
    perplex_thred=20,
    # number of perturbations to return
    num_perturbations=3,
    # the function also takes in additional arguments for huggingface generators.
    num_beams=3
)
perturbations

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


['Not is great for kids.', 'It is not for kids.', 'It is not great for kids.']

In [8]:
# detect the ctrl code from a given sentence pair
pj.detect_ctrl_code("it's great for kids.", 'It is great for kids but not for any adults.')

'negation'

# Select for diversity

Having each perturbation be represented by its token changes, control code, and dependency tree strcuture, we greedily select the ones that are least similar to those already selected. This tries to avoid redundancy in common perturbations such as black -> white.


In [5]:
# over-generate some examples

orig_text = "It is great for kids."
perturb_texts = pj.perturb(
    orig_sent=orig_text, perplex_thred=10, num_perturbations=None, num_beams=3)
orig_and_perturb_pairs = [(orig_text, perturb_text) for perturb_text in perturb_texts]
orig_and_perturb_pairs

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

[('It is great for kids.', 'It is good for kids too.'),
 ('It is great for kids.', 'It is not great for kids.'),
 ('It is great for kids.', 'It is not for kids.'),
 ('It is great for kids.', 'It is terrible for kids.'),
 ('It is great for kids.', 'It is great for kids but not for adults.'),
 ('It is great for kids.', 'is great for kids.'),
 ('It is great for kids.', 'It is not good for kids.'),
 ('It is great for kids.', 'It is bad for kids too.'),
 ('It is great for kids.', 'It is good for kids.'),
 ('It is great for kids.', 'It is for kids.'),
 ('It is great for kids.', 'It is great for adults.'),
 ('It is great for kids.', 'It is great for any audience.'),
 ('It is great for kids.', 'It is bad for kids.')]

In [6]:
sampled = pj.select_diverse_perturbations(
    orig_and_perturb_pairs=orig_and_perturb_pairs, nsamples=3)
sampled

/Users/smcaleese/Documents/masters-thesis-code/.venv/lib/python3.12/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/smcaleese/Documents/masters-thesis-code/.venv/lib/python3.12/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


[('It is great for kids.', 'It is good for kids too.'),
 ('It is great for kids.', 'It is not great for kids.'),
 ('It is great for kids.', 'is great for kids.')]

# Select surprising perturbations as counterfactual explanations

Because different models/explainers may have different forms of predictions/feature weight computation methods, Polyjuice selection expects all predictions and feature weights to be precomputed. Here, we give an example of Quora Question Pair Detection. 


In [ ]:
# set a perturbation base
orig = (
    "How can I help a friend experiencing serious depression?",
    "How do I help a friend who is in depression?"
)
orig_label = 1

# we perturb the second question.

In [ ]:
# get a model
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
import torch
model_name = "textattack/bert-base-uncased-QQP"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
# sentiment analysis is a general name in Huggingface to load the pipeline for text classification tasks.
# set device=-1 if you don't have a gpu
pipe = pipeline(
    "sentiment-analysis", model=model, tokenizer=tokenizer, 
    framework="pt", device=0 if is_cuda else -1, return_all_scores=True)

In [ ]:
# some wrapper for prediction
import numpy as np
def extract_predict_label(raw_pred):
    raw_pred = sorted(raw_pred, key=lambda r: -r["score"])
    if raw_pred:
        return raw_pred[0]["label"]
    return None
def predict(examples, predictor, batch_size=128):
    raw_preds, preds, distribution = [], [], []
    with torch.no_grad():
        for e in (range(0, len(examples), batch_size)):
            raw_preds.extend(predictor(examples[e:e+batch_size]))
    for raw_pred in raw_preds:
        raw_pred = raw_pred if type(raw_pred) == list else [raw_pred]
        for m in raw_pred:
            m["label"] = int(m["label"].split("_")[1])
    return raw_preds

p = predict([orig], predictor=pipe)[0]
(p, extract_predict_label(p))

In [ ]:
## collect some base perturbations
from polyjuice.generations import ALL_CTRL_CODES

# perturb the second question in orig.
perturb_idx = 1
perturb_texts = pj.perturb(
    orig[perturb_idx], 
    ctrl_code=ALL_CTRL_CODES, 
    num_perturbations=None, perplex_thred=10)

perturb_texts


In [ ]:
# To estimate feature importance, we set up shap explainer
# install shap
!pip install shap

In [ ]:
import shap
import functools
from copy import deepcopy
# setup a prediction function for computing the shap feature importance

def wrap_perturbed_instances(perturb_texts, orig, perturb_idx=1):
    perturbs = []
    for a in perturb_texts:
        curr_example = deepcopy(list(orig))
        curr_example[perturb_idx] = a
        perturbs.append(tuple(curr_example))
    return perturbs

def predict_on_perturbs(perturb_texts, orig, predictor, perturb_idx=1):
    perturbs = wrap_perturbed_instances(perturb_texts, orig, perturb_idx)
    perturbs_preds = predict(perturbs, predictor=predictor)
    perturbs_pred_dicts = [{p["label"]: p["score"] for p in perturbs_pred} for perturbs_pred in perturbs_preds]
    orig_preds = predict([orig], predictor=predictor)
    orig_pred = extract_predict_label(orig_preds[0])
    # the return is probability of the originally predicted label
    return [pr_dict[orig_pred] for pr_dict in perturbs_pred_dicts]
def normalize_shap_importance(features, importances, is_use_abs=True):
    normalized_features = {}
    for idx, (f, v) in enumerate(zip(features, importances)):
        f = f.strip('Ġ')
        if not f.startswith("##"): 
            key, val = "", 0
        key += f.replace("#", "").strip()
        val += v
        if (idx == len(features)-1 or (not features[idx+1].startswith("##"))) and key != "":
            normalized_features[key] = abs(val) if is_use_abs else val
    return normalized_features
def explain_with_shap(orig, predictor=pipe, tokenzier=pipe.tokenizer, perturb_idx=1):
    predict_for_shap_func = functools.partial(
        predict_on_perturbs, orig=orig, predictor=predictor, perturb_idx=perturb_idx)
    shap_explainer = shap.Explainer(predict_for_shap_func, tokenizer) 
    exp = shap_explainer([str(orig[perturb_idx])])
    return normalize_shap_importance(exp.data[0], exp.values[0])

feature_importance_dict = explain_with_shap(orig)
feature_importance_dict

In [ ]:
# get the predictions for original and also new instances
orig_pred = predict([orig], predictor=pipe)[0]

perturb_instances = wrap_perturbed_instances(perturb_texts, orig, perturb_idx)
perturb_preds = predict(perturb_instances, predictor=pipe)

surprises = pj.select_surprise_explanations(
    orig_text=orig[perturb_idx], 
    perturb_texts=perturb_texts, 
    orig_pred=orig_pred, 
    perturb_preds=perturb_preds, 
    feature_importance_dict=feature_importance_dict)
surprises